In [1]:
import os
import pandas as pd
import numpy as np
import json
import folium
from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import branca.colormap as cm
from matplotlib import colors

In [2]:
df_refugee = pd.read_csv("time_series.csv",low_memory=False)
display(df_refugee.size)
df_refugee['Value'] = pd.to_numeric(df_refugee['Value'], errors='coerce')
df_refugee= df_refugee[df_refugee['Value'] > 0]
df_refugee = df_refugee[df_refugee['Year']>1984]
df_refugee = df_refugee.groupby(['Year','Country / territory of asylum/residence','Origin']).sum().reset_index()
df_refugee = df_refugee.rename(columns={"Year":"year","Value":"value"})

1492205

In [3]:
df_geo  = pd.read_csv("geo.csv",na_values ='NaN',keep_default_na =False)
df_merge = df_refugee.merge(df_geo, left_on='Country / territory of asylum/residence', right_on='name', how='inner')
df_merge = df_merge[['year','value','Origin',"country"]]
df_merge = df_merge.rename(columns={"country" : "country_asylum"})
df_merge = df_merge.merge(df_geo, left_on='Origin', right_on='name', how='inner')
df_merge = df_merge[['year',"country","country_asylum",'value']]
df_merge = df_merge.rename(columns={"country":"country_origin"})
df_merge = df_merge.sort_values("year")

display(df_merge.head())

,year,country_origin,country_asylum,value
0,1985,CL,DZ,100.0
45149,1985,AF,SA,9000.0
418,1985,CL,RS,90.0
419,1985,CL,XK,90.0
66751,1985,PL,RS,200.0


In [4]:
df_immigration_exit = df_merge.drop("country_asylum",1)
df_immigration_exit = df_immigration_exit.groupby(["year","country_origin"]).sum().reset_index()
df_immigration_exit= df_immigration_exit.sort_values(["country_origin","year"])
display(df_immigration_exit.head())
df_immigration_exit.to_csv("data_immigration_exit.csv", sep=',', encoding='utf-8',index = False)

,year,country_origin,value
894,1994,AD,2.0
1061,1995,AD,2.0
1233,1996,AD,3.0
1410,1997,AD,1.0
1589,1998,AD,2.0


In [5]:
df_immigration_entry = df_merge.drop("country_origin",1)
df_immigration_entry = df_immigration_entry.groupby(["year","country_asylum"]).sum().reset_index()
df_immigration_entry= df_immigration_entry.sort_values(["country_asylum","year"])
df_immigration_entry = df_immigration_entry[["country_asylum","year","value"]]
display(df_immigration_entry.head())
df_immigration_entry.to_csv("data_immigration_entry.csv", sep=',', encoding='utf-8',index = False)

,country_asylum,year,value
0,AE,1985,6350.0
620,AE,1993,427.0
754,AE,1994,474.0
888,AE,1995,404.0
1024,AE,1996,450.0


In [6]:
import math
def keepNonNan(country_origin,country_asylum):
    if (country_origin == 0):
        return country_asylum
    if (country_asylum == 0):
        return country_origin 
    return country_origin

In [7]:
df_immigration_delta = df_immigration_entry.merge(df_immigration_exit,right_on=['year','country_origin'], left_on=['year','country_asylum'], how='outer')
df_immigration_delta = df_immigration_delta.fillna(0)
df_immigration_delta['country'] = df_immigration_delta.apply(lambda x:keepNonNan(x['country_origin'],x['country_asylum']),axis = 1)

df_immigration_delta = df_immigration_delta[df_immigration_delta.country != 0]
df_immigration_delta['value'] = df_immigration_delta.value_x - df_immigration_delta.value_y

df_immigration_delta = df_immigration_delta[["year","country","value"]]
df_immigration_delta = df_immigration_delta.sort_values(["year","country"])
display(df_immigration_delta.head())

,year,country,value
0,1985,AE,6350.0
4355,1985,AF,-4654343.0
4380,1985,AL,-3067.0
4387,1985,AM,-333.0
104,1985,AO,-226502.0


In [8]:
df_immigration_delta.to_csv("data_immigration_delta.csv", sep=',', encoding='utf-8',index = False)